<div align="center">
    <h1> 🎯 LUMIOS 🎯</h1>
    <h3>Label Using Machine In Organic Samples</h3>
</div>




## Notebook Introduction ✍️

This notebook presents the code used to transform molecules (which are in a text-based DataFrame) into images, with the aim of building a Convolutional Neural Network (CNN) for a specific application.



<table align="left" class="dashed-box">
<tr>
    <td>💡</td>
    <td><td>The LUMIOS ARTICLE is under review by the journal <b>EXPERT SYSTEMS WITH APPLICATIONS</b>. DOI: <a href="http://dx.doi.org/10.2139/ssrn.4341603">http://dx.doi.org/10.2139/ssrn.4341603</a></td>
</td>
</tr>
</table><br/><br/><br>


## About the developer: 
### 👨🏽‍🏫 RAFAEL VIEIRA 
- Professor at the Instituto Federal de Rondônia, campus de Ji-Paraná. 

<table align="left" class="dashed-box">
<tr>
    <td>💡</td>
    <td><td>"Throughout my journey, I have always sought to integrate the richness of natural chemistry with promising Artificial Intelligence techniques, such as Machine Learning and Deep Learning. From this fusion, innovative solutions and discoveries have emerged."</td>
</td>
</tr>
</table><br/><br/><br/>

##  💾 1. Data
#### The Dataset (Referred to as Chemical Space) Was Obtained from Two Main Platforms:
- CORTELLIS (Where "DRUG" Data Was Obtained)
- LOTUS (Natural Product Database)

In [1]:
import pandas as pd
df = pd.read_csv('/Users/rafaelvieira/Downloads/LUMIOS - Notebook/descritores_PN_respiratory_13396.csv')
df.head()

,Entry Number,smiles,type,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,68765,[H]OC(=NC([H])(C([H])([H])[H])C([H])([H])OC([H...,PN,10.164502,-0.445099,10.164502,0.030833,0.212838,387.564,350.268,...,0,0,0,0,0,0,0,0,0,0
1,23939,[H]OC(=O)C([H])=C([H])C(=O)C([H])([H])C([H])([...,PN,11.810571,-1.257729,11.810571,0.047361,0.350179,392.404,368.212,...,0,0,0,0,0,0,0,0,0,0
2,49184,[H]OC(=O)C1([H])N([H])C(=O)C([H])(C([H])([H])[...,PN,14.635249,-1.880928,14.635249,0.009939,0.079820,1048.248,974.664,...,0,0,0,0,0,0,0,0,0,0
3,24167,[H]OC12c3c([H])c([H])c([H])c([H])c3N([H])C1([H...,PN,10.295694,-0.795046,10.295694,0.245463,0.621942,177.203,166.115,...,0,0,0,0,0,0,0,0,0,0
4,51906,[H]Oc1c([H])c([H])c(C([H])([H])C2([H])C(=O)N([...,PN,15.025844,-1.826261,15.025844,0.062522,0.081104,1051.274,976.682,...,0,0,0,0,0,0,0,0,0,0


<table align="left" class="dashed-box">
<tr>
    <td>💡</td>
    <td><td>For this notebook, only the SMILES (Simplified Molecular-Input Line-Entry System) will be used to generate images."</td>
</td>
</tr>
</table><br/><br/><br/>

### 1.1 - Duplicates removal 🪞

In [2]:
df = df.drop_duplicates(subset=['smiles'], keep = 'last')

In [3]:
df.shape

(13396, 211)


<table align="left" class="dashed-box">
<tr>
    <td>💡</td>
    <td><td>There are <b>13,396 molecules</b> in the dataset. Since it is a binary classification, we chose to have an equal amount of data in each class</td>
</td>
</tr>
</table><br/><br/><br/>

In [4]:
df['type'].unique()

array(['PN', 'Drug'], dtype=object)

In [6]:
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
import pandas as pd

# Conte as ocorrências de cada classe
class_counts = df['type'].value_counts().reset_index()
class_counts.columns = ['type', 'count']

# Mapeie as cores para as classes
colors = {'PN': 'crimson', 'Drug': 'lightblue'}
class_counts['colors'] = class_counts['type'].map(colors)

# Crie o gráfico de barras com as cores mapeadas
# fig = px.bar(class_counts, x='type', y='count', title='Classes', color='colors')

fig = go.Figure(data=[
    go.Bar(
        x=class_counts['type'],
        y=class_counts['count'],
        text=class_counts['count'],
        marker=dict(color=class_counts['colors']),
    )
])

# Mostre o gráfico
fig.show()

In [7]:
df['type'] = df['type'].replace('PN', 'NP')

In [8]:
respiratory = df.loc[df['type'] == 'Drug']
NP = df.loc[df['type']=='NP']

In [9]:
print(len(respiratory))
print(len(NP))

6741
6655


In [10]:
test_size = 1-(len(NP)/len(respiratory))

### 1.2 - Undersampling
Adjusting the same amount of data for each class

In [11]:
## Dessa forma, iremos filtrar para 63803 moleculas de plantas também
from sklearn.model_selection import train_test_split

respiractory_undersampling_use, respiractory_undersampling_descart = train_test_split(respiratory, test_size= test_size, random_state=42,
                                                                          shuffle=True)

In [12]:
len(respiractory_undersampling_use)

6654

In [13]:
drugs = respiractory_undersampling_use

In [14]:
print(len(drugs))
print(len(NP))

6654
6655


### 1.2 - Converting SMILES to image 💬➡️➡️⚛️

In [15]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from rdkit.Chem import Draw
from itertools import combinations
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors

In [16]:
NP.reset_index(inplace=True, drop=True)

In [17]:
NP

,Entry Number,smiles,type,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,68765,[H]OC(=NC([H])(C([H])([H])[H])C([H])([H])OC([H...,NP,10.164502,-0.445099,10.164502,0.030833,0.212838,387.564,350.268,...,0,0,0,0,0,0,0,0,0,0
1,23939,[H]OC(=O)C([H])=C([H])C(=O)C([H])([H])C([H])([...,NP,11.810571,-1.257729,11.810571,0.047361,0.350179,392.404,368.212,...,0,0,0,0,0,0,0,0,0,0
2,49184,[H]OC(=O)C1([H])N([H])C(=O)C([H])(C([H])([H])[...,NP,14.635249,-1.880928,14.635249,0.009939,0.079820,1048.248,974.664,...,0,0,0,0,0,0,0,0,0,0
3,24167,[H]OC12c3c([H])c([H])c([H])c([H])c3N([H])C1([H...,NP,10.295694,-0.795046,10.295694,0.245463,0.621942,177.203,166.115,...,0,0,0,0,0,0,0,0,0,0
4,51906,[H]Oc1c([H])c([H])c(C([H])([H])C2([H])C(=O)N([...,NP,15.025844,-1.826261,15.025844,0.062522,0.081104,1051.274,976.682,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6650,37267,[H]OC([H])(C([H])([H])[H])C1([H])OC(=O)C(C2([H...,NP,11.234537,-0.656939,11.234537,0.103286,0.484710,184.191,172.095,...,0,0,0,0,0,0,0,0,0,0
6651,6289,[H]Oc1c([H])c([H])c(-c2oc3c(O[H])c(OC([H])([H]...,NP,12.859549,-0.703168,12.859549,0.020296,0.607670,360.318,344.190,...,0,0,0,0,0,0,0,0,0,0
6652,54959,[H]OC1([H])C2=C(OC([H])([H])[H])C(=O)C([H])=C(...,NP,13.375765,-1.488981,13.375765,0.079292,0.281608,576.643,536.323,...,0,0,0,0,0,0,0,0,0,0
6653,866,[H]OC(=O)C([H])(C([H])([H])[H])C([H])([H])C(=O...,NP,12.789495,-1.028091,12.789495,0.010090,0.600967,401.459,374.243,...,0,0,0,0,0,0,0,0,0,0


### 1.3 - Saving NP in image! 🍃 ⇾ ⚛️

In [18]:
import os
from PIL import Image
from rdkit import Chem
from rdkit.Chem import Draw

# Verifique se a pasta "imagens" existe e, se não existir, crie-a
if not os.path.exists("images"):
    os.makedirs("images")

# Verifique se a pasta "imagens/drugs" existe e, se não existir, crie-a
if not os.path.exists("images/NP"):
    os.makedirs("images/NP")

for i in range(0, len(NP)):
    ms = [Chem.MolFromSmiles(NP['smiles'][i])]
    img = Draw.MolsToGridImage(ms, molsPerRow=1, maxMols=1, subImgSize=(100, 100), returnPNG=False)
    nome = f"NP_{i}.png"

    # Salve a imagem na pasta "imagens/drugs"
    img.save(os.path.join("images", "NP", nome))


### 1.4 - Saving Drugs in image! 💊 ⇾ ⚛️

In [19]:
drugs.reset_index(inplace=True)

In [20]:
import os
from PIL import Image
from rdkit import Chem
from rdkit.Chem import Draw

# Verifique se a pasta "imagens" existe e, se não existir, crie-a
if not os.path.exists("images"):
    os.makedirs("images")

# Verifique se a pasta "imagens/drugs" existe e, se não existir, crie-a
if not os.path.exists("images/drugs"):
    os.makedirs("images/drugs")

for i in range(0, len(drugs)):
    ms = [Chem.MolFromSmiles(drugs['smiles'][i])]
    img = Draw.MolsToGridImage(ms, molsPerRow=1, maxMols=1, subImgSize=(100, 100), returnPNG=False)
    nome = f"drug_{i}.png"

    # Salve a imagem na pasta "imagens/drugs"
    img.save(os.path.join("images", "drugs", nome))


RDKit ERROR: [09:13:29] Explicit valence for atom # 20 N, 4, is greater than permitted
[09:13:29] Explicit valence for atom # 20 N, 4, is greater than permitted


<table align="left" class="dashed-box">
<tr>
    <td>💡</td>
    <td>Now we have two classes of molecules: NP (Natural Products) and Drugs. Sufficient material to start feeding a convolutional neural network</td>
</tr>
</table>


In [49]:
%%html
<style>
.dashed-box {
    border: 1px dashed black !important;
    font-size: 14px !important; /* 
}

.dashed-box table {

}

.dashed-box tr {
    background-color: white !important;
}

.alt-tab {
    background-color: black;
    color: #ffc351;
    padding: 4px;
    font-size: 1em;
    font-weight: bold;
    font-family: monospace;
}
// add your CSS styling here
</style>
